# Real Satellite Data API Fetch for Kovilpatti LULC Prediction

This notebook demonstrates how to fetch real satellite LULC (Land Use Land Cover) data for the Kovilpatti region using API calls. **No manual downloads required!**

## What You'll Learn

1. How to fetch real LULC data from Microsoft Planetary Computer
2. How to use ESA WorldCover as a fallback source
3. How to preprocess the data for model training
4. How to create training datasets
5. How to verify data quality
6. How to train models on real satellite data

## 1. Setup & Installation

First, let's install the required packages and import libraries.

In [ ]:
# Install required packages (uncomment if needed)
# !pip install pystac-client planetary-computer rasterio stackstac xarray numpy matplotlib

In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

# Add src to path
sys.path.insert(0, str(Path.cwd().parent))

from src.real_data.api_fetcher import KovilpattiDataFetcher, fetch_kovilpatti_data
from src.real_data.worldcover_api import WorldCoverFetcher
from src.real_data.preprocessor import RealDataPreprocessor, preprocess_for_model
from src.real_data.utils import (
    visualize_real_data, 
    print_data_summary,
    get_kovilpatti_bbox,
    calculate_real_transition_matrix
)

print("✅ All imports successful!")

## 2. Fetch Real Data from API

Now let's fetch real LULC data for Kovilpatti from Microsoft Planetary Computer.

### 2.1 Set Up Region Parameters

In [ ]:
# Kovilpatti region parameters
REGION_NAME = "Kovilpatti"
LATITUDE = 9.17  # degrees N
LONGITUDE = 77.87  # degrees E
RADIUS_KM = 10  # kilometers
YEARS = [2020, 2021, 2022]  # Years to fetch

# Display region info
bbox = get_kovilpatti_bbox()
print(f"📍 Region: {REGION_NAME}")
print(f"🌍 Center: ({LATITUDE}°N, {LONGITUDE}°E)")
print(f"📏 Radius: {RADIUS_KM} km")
print(f"📦 Bounding Box: {bbox}")
print(f"📅 Years: {YEARS}")

### 2.2 Initialize Data Fetcher

In [ ]:
# Initialize fetcher
fetcher = KovilpattiDataFetcher(
    region_name=REGION_NAME,
    lat=LATITUDE,
    lon=LONGITUDE,
    radius_km=RADIUS_KM,
    cache_dir="../data/cache"
)

print("\n✅ Fetcher initialized!")

### 2.3 Fetch Data (This may take a few minutes)

In [ ]:
# Fetch temporal sequence
print("🛰️ Fetching data from Planetary Computer...\n")
years_data = fetcher.fetch_temporal_sequence(YEARS)

if years_data:
    print(f"\n✅ Successfully fetched {len(years_data)} years!")
    for year in years_data.keys():
        print(f"  - {year}: {years_data[year].shape}")
else:
    print("\n⚠️ No data fetched. Trying fallback...")
    
    # Try WorldCover as fallback
    wc_fetcher = WorldCoverFetcher(
        region_name=REGION_NAME,
        lat=LATITUDE,
        lon=LONGITUDE,
        radius_km=RADIUS_KM,
        cache_dir="../data/cache"
    )
    years_data = wc_fetcher.fetch_temporal_sequence([2020, 2021])
    
    if years_data:
        print(f"\n✅ Fetched {len(years_data)} years from WorldCover!")

### 2.4 Visualize Downloaded Raw Data

In [ ]:
# Visualize each year
for year, data in years_data.items():
    print(f"\n{'='*60}")
    print(f" Year {year}")
    print(f"{'='*60}")
    
    # Print summary
    print_data_summary(data, f"Raw LULC Data - {year}")
    
    # Visualize
    plt.figure(figsize=(12, 10))
    plt.imshow(data, cmap='tab20', interpolation='nearest')
    plt.colorbar(label='LULC Class', shrink=0.7)
    plt.title(f'Raw LULC Data - Kovilpatti {year}', fontsize=14, fontweight='bold')
    plt.xlabel('Pixel X')
    plt.ylabel('Pixel Y')
    plt.tight_layout()
    plt.show()
    
    # Class distribution
    unique, counts = np.unique(data, return_counts=True)
    percentages = (counts / counts.sum()) * 100
    
    plt.figure(figsize=(10, 4))
    plt.bar(range(len(unique)), percentages, color='steelblue')
    plt.xlabel('LULC Class')
    plt.ylabel('Percentage (%)')
    plt.title(f'Class Distribution - {year}')
    plt.xticks(range(len(unique)), unique)
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()

## 3. Preprocessing

Now let's preprocess the raw data to make it suitable for model training.

### 3.1 Class Remapping

We need to map the raw LULC classes to our standard 7 classes.

In [ ]:
# Initialize preprocessor
preprocessor = RealDataPreprocessor(n_classes=7)

# Remap one year as example
year_example = list(years_data.keys())[0]
raw_data = years_data[year_example]

print(f"\n🗺️ Remapping classes for {year_example}...")
remapped = preprocessor.remap_classes(raw_data, source="io")

# Visualize before and after
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Before
im1 = axes[0].imshow(raw_data, cmap='tab20', interpolation='nearest')
axes[0].set_title(f'Before Remapping - {year_example}', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Pixel X')
axes[0].set_ylabel('Pixel Y')
plt.colorbar(im1, ax=axes[0], label='Original Class')

# After
im2 = axes[1].imshow(remapped, cmap='tab10', interpolation='nearest', vmin=0, vmax=6)
axes[1].set_title(f'After Remapping - {year_example}', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Pixel X')
axes[1].set_ylabel('Pixel Y')
cbar = plt.colorbar(im2, ax=axes[1], label='Remapped Class', ticks=range(7))
cbar.ax.set_yticklabels(['Urban', 'Forest', 'Agriculture', 'Water', 'Barren', 'Wetland', 'Grassland'])

plt.tight_layout()
plt.show()

print(f"\n✅ Remapping complete!")
print(f"Original classes: {np.unique(raw_data)}")
print(f"Remapped classes: {np.unique(remapped)}")

### 3.2 Patch Creation

Large images are divided into smaller 256×256 patches for training.

In [ ]:
# Create patches from one year
patches = preprocessor.crop_to_patches(remapped, patch_size=256, overlap=0)

print(f"\n✂️ Created {len(patches)} patches of size 256×256")

# Visualize some patches
n_show = min(6, len(patches))
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i in range(n_show):
    axes[i].imshow(patches[i], cmap='tab10', interpolation='nearest', vmin=0, vmax=6)
    axes[i].set_title(f'Patch {i+1}', fontsize=10)
    axes[i].axis('off')

plt.suptitle('Sample Patches (256×256)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### 3.3 Quality Checks

In [ ]:
# Check data quality
print("\n🔍 Quality Checks:")
print(f"\n1. Data Shapes:")
for year, data in years_data.items():
    print(f"   {year}: {data.shape}")

print(f"\n2. Value Ranges:")
for year, data in years_data.items():
    print(f"   {year}: [{data.min()}, {data.max()}]")

print(f"\n3. Data Types:")
for year, data in years_data.items():
    print(f"   {year}: {data.dtype}")

print(f"\n4. Missing Data:")
for year, data in years_data.items():
    n_nan = np.isnan(data).sum() if np.issubdtype(data.dtype, np.floating) else 0
    print(f"   {year}: {n_nan} NaN values")

print("\n✅ Quality checks complete!")

### 3.4 Temporal Alignment

Ensure all years have consistent spatial dimensions.

In [ ]:
# Check if all years have same shape
shapes = [data.shape for data in years_data.values()]
all_same = len(set(shapes)) == 1

print(f"\n📐 Temporal Alignment Check:")
print(f"   All years same shape: {all_same}")

if not all_same:
    print(f"   Shapes: {shapes}")
    print(f"   ⚠️ Will crop to minimum dimensions during preprocessing")
else:
    print(f"   Shape: {shapes[0]}")
    print(f"   ✅ All years aligned!")

## 4. Dataset Creation

Create training samples from the multi-year data.

### 4.1 Create Temporal Samples

In [ ]:
# Create temporal samples
print("\n📦 Creating temporal training samples...")

splits = preprocess_for_model(
    years_data,
    output_dir="../data/Kovilpatti_LULC_Real",
    patch_size=256,
    source="io"  # or "esa" if using WorldCover
)

if splits:
    print("\n✅ Dataset created successfully!")
else:
    print("\n❌ Dataset creation failed!")

### 4.2 Dataset Statistics

In [ ]:
if splits:
    print("\n📊 Dataset Statistics:")
    print("\n" + "="*60)
    
    for split_name in ['train', 'val', 'test']:
        if split_name in splits:
            n_samples = len(splits[split_name]['inputs'])
            input_shape = splits[split_name]['inputs'][0].shape
            target_shape = splits[split_name]['targets'][0].shape
            
            print(f"\n{split_name.upper()}:")
            print(f"  Samples: {n_samples}")
            print(f"  Input shape: {input_shape}")
            print(f"  Target shape: {target_shape}")
    
    print("\n" + "="*60)

### 4.3 Visualize Samples

In [ ]:
if splits and 'train' in splits:
    # Show a few training samples
    n_show = min(3, len(splits['train']['inputs']))
    
    for i in range(n_show):
        input_seq = splits['train']['inputs'][i]  # Shape: (n_timesteps, H, W)
        target = splits['train']['targets'][i]     # Shape: (H, W)
        
        n_timesteps = input_seq.shape[0]
        
        fig, axes = plt.subplots(1, n_timesteps + 1, figsize=(4*(n_timesteps+1), 4))
        
        # Show each input timestep
        for t in range(n_timesteps):
            axes[t].imshow(input_seq[t], cmap='tab10', interpolation='nearest', vmin=0, vmax=6)
            axes[t].set_title(f'Input T-{n_timesteps-t}', fontsize=10)
            axes[t].axis('off')
        
        # Show target
        im = axes[-1].imshow(target, cmap='tab10', interpolation='nearest', vmin=0, vmax=6)
        axes[-1].set_title('Target (Prediction)', fontsize=10, fontweight='bold')
        axes[-1].axis('off')
        
        # Add colorbar
        cbar = plt.colorbar(im, ax=axes[-1], ticks=range(7), shrink=0.8)
        cbar.ax.set_yticklabels(['Urban', 'Forest', 'Agri', 'Water', 'Barren', 'Wetland', 'Grass'], fontsize=8)
        
        plt.suptitle(f'Training Sample {i+1}', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()

## 5. Verify Data

Load the saved data to verify it's in the correct format.

In [ ]:
# Load saved data
data_dir = Path("../data/Kovilpatti_LULC_Real")

print("\n🔍 Verifying saved data...")
print(f"\nDirectory: {data_dir}")

# Check files
required_files = [
    'train_inputs.npy', 'train_targets.npy',
    'val_inputs.npy', 'val_targets.npy',
    'test_inputs.npy', 'test_targets.npy',
    'metadata.txt'
]

print("\nFiles:")
for filename in required_files:
    filepath = data_dir / filename
    exists = filepath.exists()
    size_mb = filepath.stat().st_size / (1024**2) if exists else 0
    status = "✅" if exists else "❌"
    print(f"  {status} {filename:20s} ({size_mb:.2f} MB)")

# Load and verify shapes
print("\nData Shapes:")
for split in ['train', 'val', 'test']:
    try:
        inputs = np.load(data_dir / f"{split}_inputs.npy")
        targets = np.load(data_dir / f"{split}_targets.npy")
        print(f"  {split:5s}: inputs={inputs.shape}, targets={targets.shape}")
    except:
        print(f"  {split:5s}: ❌ Failed to load")

# Show metadata
metadata_file = data_dir / "metadata.txt"
if metadata_file.exists():
    print("\nMetadata:")
    with open(metadata_file, 'r') as f:
        for line in f:
            print(f"  {line.strip()}")

print("\n✅ Verification complete!")

## 6. Calculate Transition Matrix

Analyze actual land cover transitions from the real data.

In [ ]:
# Calculate transition matrix from real data
if len(years_data) >= 2:
    print("\n🔄 Calculating transition matrix from real data...")
    
    # First remap all years
    remapped_years = {}
    for year, data in years_data.items():
        remapped_years[year] = preprocessor.remap_classes(data, source="io")
    
    # Calculate transitions
    transition_matrix = calculate_real_transition_matrix(remapped_years, n_classes=7)
    
    # Visualize
    class_names = ['Urban', 'Forest', 'Agriculture', 'Water', 'Barren', 'Wetland', 'Grassland']
    
    plt.figure(figsize=(10, 8))
    im = plt.imshow(transition_matrix, cmap='YlOrRd', interpolation='nearest', vmin=0, vmax=1)
    
    # Add colorbar
    cbar = plt.colorbar(im, label='Transition Probability')
    
    # Add labels
    plt.xticks(range(7), class_names, rotation=45, ha='right')
    plt.yticks(range(7), class_names)
    plt.xlabel('To Class', fontsize=12)
    plt.ylabel('From Class', fontsize=12)
    plt.title('Land Cover Transition Matrix (Real Data)', fontsize=14, fontweight='bold')
    
    # Add text annotations
    for i in range(7):
        for j in range(7):
            text = plt.text(j, i, f'{transition_matrix[i, j]:.2f}',
                          ha="center", va="center", color="black", fontsize=8)
    
    plt.tight_layout()
    plt.show()
    
    print("\n✅ Transition matrix calculated and visualized!")
else:
    print("\n⚠️ Need at least 2 years of data for transition matrix")

## 7. Next Steps: Train on Real Data

Now that you have preprocessed real satellite data, you can train your model!

In [ ]:
print("\n" + "="*80)
print(" 🎯 NEXT STEPS")
print("="*80)

print("\n1. Your data is ready at: data/Kovilpatti_LULC_Real/")
print("\n2. Train your model using:")
print("   python scripts/run_training_real.py --data_dir data/Kovilpatti_LULC_Real/ --real_data")
print("\n3. Or define your model here in the notebook and train interactively")

print("\n" + "="*80)
print(" ✨ CONGRATULATIONS! ✨")
print("="*80)
print("\nYou've successfully:")
print("  ✅ Fetched real satellite data via API")
print("  ✅ Preprocessed data for model training")
print("  ✅ Created train/val/test splits")
print("  ✅ Verified data quality")
print("  ✅ Analyzed land cover transitions")
print("\nYour LULC prediction model is ready to train on REAL satellite data!")
print("\n" + "="*80)

## Summary

In this notebook, you learned how to:

1. **Fetch real satellite data** from Microsoft Planetary Computer and ESA WorldCover APIs
2. **Preprocess the data** including class remapping and patch creation
3. **Create temporal training samples** from multi-year data
4. **Verify data quality** and format
5. **Analyze land cover transitions** from real observations

### Key Advantages of Using Real Data:

- **Realistic patterns**: Actual land cover distributions and transitions
- **Ground truth**: Validated satellite observations
- **Generalization**: Models trained on real data work better on real predictions
- **Scientific validity**: Results can be published and trusted

### Resources:

- [Microsoft Planetary Computer](https://planetarycomputer.microsoft.com/)
- [ESA WorldCover](https://esa-worldcover.org/)
- [Project Documentation](../docs/REAL_DATA_GUIDE.md)

Happy modeling! 🚀